In [1]:
# Importing packages

import pandas as pd
import numpy  as np
from sklearn.cluster import KMeans
import seaborn as sns
import datetime 
import chardet


# tBaustoff Dataset

The tBaustoff dataset provides material mapping information with 10 columns. It connects to the ÖKOBAUDAT dataset through process UUIDs and contains end-of-life scenario information for various materials.

End-of-Life Scenarios: Different materials have different potential pathways

- Reuse: Direct application in new construction (highest value retention)
- Recycling: Processing into new material (partial value retention)
- Downcycling: Processing into lower-value applications
- Disposal: Landfilling or incineration (lowest circularity value)

## Data Wrangling

### Data Extraction

In [2]:
# detect files encoding
with open("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/OBD_2024_I.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Read first 100,000 bytes
    print(result)
# Importing data from CSV and converting to dataframe
obd_df = pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/obd_merged.csv", delimiter=";", encoding = "utf-8-sig", low_memory=False)
tbau_df = pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/tBaustoff_with_OBD_mapping.csv", delimiter=",", encoding = result["encoding"], low_memory=False)


{'encoding': 'ISO-8859-1', 'confidence': 0.7295400999999999, 'language': ''}


In [3]:
tbau_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   oekobaudatProcessUuid        728 non-null    object 
 1   oekobaudatDatastockUuid      731 non-null    object 
 2   oekobaudatDatastockName      731 non-null    object 
 3   productId                    815 non-null    int64  
 4   tBaustoffVersion             815 non-null    object 
 5   productName                  815 non-null    object 
 6   eolCategoryId                815 non-null    int64  
 7   eolCategoryName              815 non-null    object 
 8   eolScenarioUnbuiltReal       815 non-null    object 
 9   eolScenarioUnbuiltPotential  815 non-null    object 
 10  technologyFactor             815 non-null    float64
dtypes: float64(1), int64(2), object(8)
memory usage: 70.2+ KB


In [6]:
tbau_df.head()

,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.00
1,0dec19c7-53db-4cb9-bbba-4d44d3da62a9,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.00
2,bd6d6d89-b76d-4002-a217-afffbb8aa308,448d1096-2017-4901-a560-f652a83c737e,OBD_2020_II,2,2024-Q4,Aluminium Profil,2,Alu unbeschichtet od. eloxiert (Sz. Knetleg zu...,RC+,CL+,0.75
3,fdc99ab8-d843-44ec-a66c-92367d244321,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,2,2024-Q4,Aluminium Profil,2,Alu unbeschichtet od. eloxiert (Sz. Knetleg zu...,RC+,CL+,0.75
4,3feca796-791b-46d3-8160-95ef243ffb9d,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,2,2024-Q4,Aluminium Profil,2,Alu unbeschichtet od. eloxiert (Sz. Knetleg zu...,RC+,CL+,0.75


### Data Cleaning

In [4]:
# identifying the columns with missing values
missing_values = tbau_df.isnull().sum()

In [5]:
missing_values

oekobaudatProcessUuid          87
oekobaudatDatastockUuid        84
oekobaudatDatastockName        84
productId                       0
tBaustoffVersion                0
productName                     0
eolCategoryId                   0
eolCategoryName                 0
eolScenarioUnbuiltReal          0
eolScenarioUnbuiltPotential     0
technologyFactor                0
dtype: int64

In [ ]:
# unique values in the 'OBD' column
tbau_df["oekobaudatProcessUuid"].nunique()

728

In [8]:
len(tbau_df["oekobaudatProcessUuid"])


815

815 - 728  = 87 
Each uuid is unique (not like in obd). The remaining are missing values
Note that data entries in `ÖKOBAUDAT` are referred to as *Processes*, not materials, as life cycle assessment is conducted at the process level. Since we are talking of *ProcessUuid* in connection with `ÖKOBAUDAT`, it means that the data is related to a speficic material combined with assessments through its entire life cycle.